Suman Kumar Sahu

Objective:
Dataset:

Status:

Highlevel Abstraction: Estimators and Transformers

DataFrame->Transformer(conversion of dataframes)->Estimator(ML algos are all estimators). An algo that fit on dataframe to produce Tranformer (change dataframes)...

Pipeline: Chains of estimators and transformers to form a machine learning workflow
Chains a series of operations to be performed on DataFrames

Working with Spark ML Libraries

which means no sc but SparkSession which encapsulates spark context sql context and all other context available with spark


In [4]:
from pyspark.sql import SparkSession
sp=SparkSession\
.builder\
.appName('Predicting the grape variety from wine characteristic')\
.getOrCreate()

rawData=spark.read\
.format('csv')\
.option('header','false')\
.load("/home/titan/dataset/ML_JRVI/wine.data")

In [5]:
rawData

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string]

In [8]:
dataset=rawData.toDF('Label',
                    'Alcohol',
                    'MalicAcid',
                    'Ash',
                    'AshAlkalinity',
                    'Magnesium',
                    'TotalPhenols',
                    'Flavanoids',
                    'NonFlavanoidPhenols',
                    'Proanthocyanins',
                    'ColorIntensity',
                    'Hue',
                    'OD',
                    'Proline')

In [9]:
dataset

DataFrame[Label: string, Alcohol: string, MalicAcid: string, Ash: string, AshAlkalinity: string, Magnesium: string, TotalPhenols: string, Flavanoids: string, NonFlavanoidPhenols: string, Proanthocyanins: string, ColorIntensity: string, Hue: string, OD: string, Proline: string]

In [12]:
dataset.take(3)

[Row(Label='1', Alcohol='14.23', MalicAcid='1.71', Ash='2.43', AshAlkalinity='15.6', Magnesium='127', TotalPhenols='2.8', Flavanoids='3.06', NonFlavanoidPhenols='.28', Proanthocyanins='2.29', ColorIntensity='5.64', Hue='1.04', OD='3.92', Proline='1065'),
 Row(Label='1', Alcohol='13.2', MalicAcid='1.78', Ash='2.14', AshAlkalinity='11.2', Magnesium='100', TotalPhenols='2.65', Flavanoids='2.76', NonFlavanoidPhenols='.26', Proanthocyanins='1.28', ColorIntensity='4.38', Hue='1.05', OD='3.4', Proline='1050'),
 Row(Label='1', Alcohol='13.16', MalicAcid='2.36', Ash='2.67', AshAlkalinity='18.6', Magnesium='101', TotalPhenols='2.8', Flavanoids='3.24', NonFlavanoidPhenols='.3', Proanthocyanins='2.81', ColorIntensity='5.68', Hue='1.03', OD='3.17', Proline='1185')]

In [13]:
dataset.show(5)

+-----+-------+---------+----+-------------+---------+------------+----------+-------------------+---------------+--------------+----+----+-------+
|Label|Alcohol|MalicAcid| Ash|AshAlkalinity|Magnesium|TotalPhenols|Flavanoids|NonFlavanoidPhenols|Proanthocyanins|ColorIntensity| Hue|  OD|Proline|
+-----+-------+---------+----+-------------+---------+------------+----------+-------------------+---------------+--------------+----+----+-------+
|    1|  14.23|     1.71|2.43|         15.6|      127|         2.8|      3.06|                .28|           2.29|          5.64|1.04|3.92|   1065|
|    1|   13.2|     1.78|2.14|         11.2|      100|        2.65|      2.76|                .26|           1.28|          4.38|1.05| 3.4|   1050|
|    1|  13.16|     2.36|2.67|         18.6|      101|         2.8|      3.24|                 .3|           2.81|          5.68|1.03|3.17|   1185|
|    1|  14.37|     1.95| 2.5|         16.8|      113|        3.85|      3.49|                .24|           2.1

In [36]:
from pyspark.ml.linalg import Vectors

def vectorize(data):
    return data.rdd.map(lambda x: [x[0],Vectors.dense(x[1:])]).toDF(['label','features'])
#note the mention of '.rdd' logically incoherent
#dense vector opposite of spars vector... sparse representation vs dense representation

In [37]:
vectorisedData=vectorize(dataset)
type(vectorisedData)

pyspark.sql.dataframe.DataFrame

In [38]:
vectorisedData.take(5)
vectorisedData.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|[14.23,1.71,2.43,...|
|    1|[13.2,1.78,2.14,1...|
|    1|[13.16,2.36,2.67,...|
|    1|[14.37,1.95,2.5,1...|
|    1|[13.24,2.59,2.87,...|
+-----+--------------------+
only showing top 5 rows



In [41]:
# converting categorical data to 


from pyspark.ml.feature import StringIndexer
labelIndexer=StringIndexer (inputCol='label',
outputCol='indexedLabel')

In [42]:
indexedData=labelIndexer.fit(vectorisedData).transform(vectorisedData)
indexedData.take(2)

[Row(label='1', features=DenseVector([14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]), indexedLabel=1.0),
 Row(label='1', features=DenseVector([13.2, 1.78, 2.14, 11.2, 100.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.4, 1050.0]), indexedLabel=1.0)]

In [43]:
indexedData

DataFrame[label: string, features: vector, indexedLabel: double]

In [44]:
indexedData.select('label').distinct().show()

+-----+
|label|
+-----+
|    3|
|    1|
|    2|
+-----+



In [45]:
indexedData.select('indexedLabel').distinct().show()

+------------+
|indexedLabel|
+------------+
|         0.0|
|         1.0|
|         2.0|
+------------+



In [48]:
(trainingData,testData)=indexedData.randomSplit([0.8,0.2])

In [58]:
from pyspark.ml.classification import DecisionTreeClassifier

In [63]:
dtree= DecisionTreeClassifier(labelCol='indexedLabel',
                                   featuresCol='features',
                                   impurity='gini',
                                   maxDepth=3,
                                   )

In [62]:
# help(DecisionTreeClassifier)

In [64]:
model1=dtree.fit(trainingData)

In [71]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator=MulticlassClassificationEvaluator(labelCol='indexedLabel',
                                           predictionCol='prediction',
                                           metricName='f1')

In [72]:
transformed_data=model1.transform(testData)
transformed_data.show(5)

+-----+--------------------+------------+--------------+--------------------+----------+
|label|            features|indexedLabel| rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------+--------------------+----------+
|    1|[12.85,1.6,2.52,1...|         1.0| [2.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|    1|[12.93,3.8,2.65,1...|         1.0| [2.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|    1|[13.05,1.65,2.55,...|         1.0|[0.0,47.0,0.0]|       [0.0,1.0,0.0]|       1.0|
|    1|[13.2,1.78,2.14,1...|         1.0|[0.0,47.0,0.0]|       [0.0,1.0,0.0]|       1.0|
|    1|[13.24,2.59,2.87,...|         1.0|[47.0,1.0,0.0]|[0.97916666666666...|       0.0|
+-----+--------------------+------------+--------------+--------------------+----------+
only showing top 5 rows



In [73]:
print(evaluator.getMetricName(),'accuracy:',
      evaluator.evaluate(transformed_data))

f1 accuracy: 0.8379691653375865
